In [72]:
import pandas as pd
import csv
import os
import numpy as np
import matplotlib.pyplot as plt

### paths

In [73]:
pathaudio = 'Data/train/audio/'
patheyes = 'Data/train/eyes/'
pathface = 'Data/train/face_nn/'
pathkinect = 'Data/train/kinect/'
pathlabels = 'Data/train/labels/'

In [74]:
list_audio = os.listdir(pathaudio)
list_eyes = os.listdir(patheyes)
list_face = os.listdir(pathface)
list_kinect = os.listdir(pathkinect)
list_labels = os.listdir(pathlabels)

In [75]:
print('Количество файлов по папкам:\n1) audio: {}'
      '\n2) eyes: {} \n3) face: {} \n4) kinect: {} \n5) labels: {}'
     .format(len(list_audio), len(list_eyes), len(list_face), len(list_kinect), len(list_labels)))

Количество файлов по папкам:
1) audio: 272
2) eyes: 306 
3) face: 256 
4) kinect: 295 
5) labels: 312


In [76]:
dict_of_labels = {}
count_dict = {}

for label in list_labels:
    count_audio = list_audio.count(label)
    count_eyes = list_eyes.count(label)
    count_face = list_face.count(label)
    count_kinect = list_kinect.count(label)
    
    if count_audio + count_eyes + count_face + count_kinect == 0:
        continue
    
    object_dict = {'audio' : count_audio, 'eyes' : count_eyes, 'face' : count_face, 'kinect' : count_kinect}
    
    dict_of_labels[label] = object_dict
    count_dict[label] = count_audio + count_eyes + count_face + count_kinect

In [77]:
print(len(dict_of_labels))

list(dict_of_labels.items())[:10]

306


[('id5181a851.csv', {'audio': 1, 'eyes': 1, 'face': 1, 'kinect': 1}),
 ('id13feb714.csv', {'audio': 1, 'eyes': 1, 'face': 1, 'kinect': 1}),
 ('idde0a7733.csv', {'audio': 0, 'eyes': 1, 'face': 0, 'kinect': 1}),
 ('id42d71a97.csv', {'audio': 0, 'eyes': 1, 'face': 1, 'kinect': 1}),
 ('id4d4cd53e.csv', {'audio': 1, 'eyes': 1, 'face': 0, 'kinect': 1}),
 ('id571e2b00.csv', {'audio': 0, 'eyes': 1, 'face': 1, 'kinect': 0}),
 ('id911ae0ab.csv', {'audio': 1, 'eyes': 1, 'face': 1, 'kinect': 1}),
 ('id8036ccb4.csv', {'audio': 1, 'eyes': 1, 'face': 1, 'kinect': 1}),
 ('idb9d6496b.csv', {'audio': 0, 'eyes': 1, 'face': 1, 'kinect': 0}),
 ('idd4bd6c5d.csv', {'audio': 1, 'eyes': 1, 'face': 1, 'kinect': 1})]

### DataRestorator

In [183]:
def restoration(data, shape, time_label, precision = 2):
    
    time_data = data[:, 0]
    set_precision = '{:.' + str(precision) + 'f}'
    step = float(set_precision.format(time_data[1] - time_data[0]))
    labelstep = 0.01
    koef = int(step / labelstep)
    new_data = []
    
    if len(time_label) != len(time_data):
             
        j = 0 # iter of time_data
        flag = 1 # flag for count passes
        for i in range(len(time_label)):

            diff = time_label[i] - time_data[j]
            if abs(diff) <= ((labelstep + 0.00001) / 2):
                if flag <= 2 * koef:
                    for _ in range(flag):
                        new_data.append(data[j])
                else:
                    #print('BIIIG PASS!!!', flag, koef)
                    new_data.append(np.zeros((flag - 1, shape[1] + 1)))
                    
                #new_data.append(data[j])
                j += 1
                flag = 1
                
            else:
                flag += 1

            
            if j == len(time_data):
                break    
                
    else:
        new_data = data
            

    new_data = np.vstack(new_data)
     
    # final tune
    if shape[0] - new_data.shape[0] < 9:
        # add last elem
        for i in range(shape[0] - new_data.shape[0]):
            new_data = np.vstack([new_data, data[-1]])                       

    elif shape[0] - new_data.shape[0] > 8:
        #print('BIG PASS at the end!!!', shape[0] - new_data.shape[0])
        new_data = np.vstack([new_data, np.zeros((shape[0] - new_data.shape[0], shape[1] + 1))])   
    
    return new_data

In [185]:
def load_lbl(lbl, path, file, labeldata, shape):
    if lbl == 1:
        datas = np.array(pd.read_csv(path + file, skiprows=[0], header=None))
        # data restore
        restore_data = restoration(datas, shape, labeldata)[:, 1:]
        # shape_assert and delete time-row
        assert restore_data.shape == shape, 'shapes dont eq'
        
    else:
        restore_data = np.zeros(shape)
        
    return restore_data

In [198]:
def data_generator(file, audio, eyes, face, kinect, dictt=None, verbose=True):    
    label_data = np.array(pd.read_csv(pathlabels + file, skiprows=[0], header=None))
    timev = label_data[:, 0]
    label_data = label_data[:, 1:]

    audio_data = load_lbl(audio, pathaudio, file, timev, (len(label_data), 36))
    eyes_data = load_lbl(eyes, patheyes, file, timev, (len(label_data), 6))
    face_data = load_lbl(face, pathface, file, timev, (len(label_data), 100))
    kinect_data = load_lbl(kinect, pathkinect, file, timev, (len(label_data), 27)) # 0.07
    
    return (label_data, audio_data, eyes_data, face_data, kinect_data)

In [188]:
for key, value in list(dict_of_labels.items())[:10]:

    data_generator(key, verbose=False, **value)

### Try

In [197]:
key, value = list(dict_of_labels.items())[135]

print(key, value)

idc469a517.csv {'audio': 1, 'eyes': 1, 'face': 1, 'kinect': 1}


In [200]:
data = data_generator(key, **value, verbose=False)

for d in data:
    print(d.shape)

(3324, 7)
(3324, 36)
(3324, 6)
(3324, 100)
(3324, 27)
